In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular, train_log, train_super_batch, train_super_batch_KL, train_super_batch_L1, train_regular_L1,train_regular_one_hot
from model_saver import model_saver, model_saver_wandb

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
torch.__version__

'1.12.1'

In [5]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [6]:
def compare_model_to_bf(model:GRUNet,raceDB:Races,example_ct):
    with torch.no_grad():
        sft_max = nn.Softmax(dim=-1)
        l_sftmax = nn.LogSoftmax(dim=-1)
        nnl_loss = nn.KLDivLoss(reduction='batchmean')
        full_test_races = raceDB.get_test_input(range(0,len(raceDB.test_race_ids)))
        full_test_races_w_prices = []
        excluded, included = 0,0
        for r in full_test_races:
            if 0 in r.prices or -1 in r.prices:
                excluded+=1
            else:
                full_test_races_w_prices.append(r)
                included+=1
        print(included,excluded)

        output = l_sftmax(model(full_test_races_w_prices))
        bf_prices = torch.log(torch.tensor([x.implied_prob for x in full_test_races_w_prices ]).to('cuda:0'))
        full_classes = torch.stack([x.classes for x in full_test_races_w_prices ])

        print()

        print(f"our loss = {nnl_loss(output,full_classes)}")
        print(f"their loss = {nnl_loss(bf_prices ,full_classes)}")
        wandb.log({"our loss":nnl_loss(output,full_classes), "their loss":nnl_loss(bf_prices ,full_classes)})

In [7]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
#dog_stats_file = open( 'new gru input 2023-01.npy', 'rb')
hidden_size = 64
raceDB = build_dataset('new_windows_gru_REAL.npy', hidden_size ,state_filter=["NSW"], margin_type='neg_raw')
raceDB.create_new_weights_v2()
# raceDB.adjust_weights({"Dapto":10, "Gunnedah":10})

(1168053, 19)
(1168053, 19)
(288467, 22)
Latest date = 2023-03-06 00:00:00
num_features_per_dog=55


  0%|          | 0/14655 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
  0%|          | 0/40482 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 40482/40482 [01:17<00:00, 523.57it/s]


number of races = 40482, number of unique dogs = 14655
0          (Dubbo, 400.0)
1          (Dubbo, 318.0)
2          (Dubbo, 318.0)
3        (Grafton, 305.0)
4        (Grafton, 305.0)
               ...       
40477    (Gosford, 388.0)
40478    (Gosford, 515.0)
40479    (Gosford, 388.0)
40480    (Gosford, 388.0)
40481    (Gosford, 388.0)
Length: 40482, dtype: object


In [8]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 32452, Test examples 8030


In [9]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file="rich-sweep-2"):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="GRU - FastTrack - AUS Testing - L1", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      model = smalll_lin_GRUNet(input_size,config['hidden_size'])
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        raceDB.fill_hidden_states_from_dict(hidden_dict=model_data['db'])
        model.load_state_dict(model_data['model_state_dict'])
        config['parent model'] = prev_model_file


      raceDB.to_cuda()

      # criterion = nn.CrossEntropyLoss(reduction='none')
      criterion = nn.SmoothL1Loss(reduction='none', beta=10)
      optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
      # optimizer = optim.Adadelta(model.parameters())
      # optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])\
      # optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9)

      print(criterion, optimizer)

      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',threshold=0.0001, patience=10000, verbose=True, factor=0.5)
      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        train_regular_L1(model, dataset, criterion, optimizer, scheduler, config)
      except KeyboardInterrupt:
        print("finished Early")
      dataset.create_hidden_states_dict()
      model_saver_wandb(model, optimizer, 450, 0.1, dataset.hidden_states_dict_gru, model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [10]:

sweep_config = {"method": "bayes"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":64}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [50]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate":{
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [100,250,500,1000]
        },
        "batch_before_backwards": {
            'values': [5,10,20]
        },

        # "batch_before_backwards": {
        #     # a flat distribution between 0 and 0.1
        #     "distribution": "uniform",
        #     "min": 3,
        #     "max": 50,
        # },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_before_backwards': {'values': [5, 10, 20]},
                'batch_size': {'values': [100, 250, 500, 1000]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [50]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 64},
                'l1_beta': {'value': 0.1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 1e-05},
                'len_data': {'value': 40482},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'bayes',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 40482},
  'hidden_size': {'value': 64},
  'epochs': {'values': [50]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-05, 'max': 0.001},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [100, 250, 500, 1000]},
  'batch_before_backwards': {'values': [5, 10, 20]}}}

In [11]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'batch_size': 5000, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.00095, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':10}

In [12]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False,prev_model_file="pious-waterfall-236")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


{'hidden_size': 64, 'batch_size': 5000, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.00095, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 10}
1000
{'hidden_size': 64, 'batch_size': 5000, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.00095, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 10}


100%|██████████| 40482/40482 [00:15<00:00, 2546.44it/s]


filled =273645
empty  =50211
0.8449588706091596null_dog=0
SmoothL1Loss() Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.00095
    maximize: False
    weight_decay: 0
)
smalll_lin_GRUNet(
  (batchnorm): BatchNorm1d(510, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=510, out_features=510, bias=True)
  (gru1): GRUCell(510, 64)
  (gru2): GRUCell(510, 64)
  (gru3): GRUCell(510, 64)
  (gru4): GRUCell(510, 64)
  (gru5): GRUCell(510, 64)
  (gru6): GRUCell(510, 64)
  (gru7): GRUCell(510, 64)
  (gru8): GRUCell(510, 64)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=512, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): Identity()
)


100%|██████████| 6/6 [00:01<00:00,  3.39it/s]6.72s/it]


...

100%|██████████| 6/6 [00:01<00:00,  3.14it/s]01.23s/it]


[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpvp78cu3awandb-media\\3baiv9fn.table.json'


100%|██████████| 6/6 [00:01<00:00,  3.50it/s]94.22s/it]


[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpvp78cu3awandb-media\\9d11wdbi.table.json'


100%|██████████| 6/6 [00:03<00:00,  1.62it/s]99.52s/it]


In [ ]:

sweep_id = wandb.sweep(sweep_config, project="GRU_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Create sweep with ID: hg9kz1xn
Sweep URL: https://wandb.ai/nickojelly/GRU_sweeps/sweeps/hg9kz1xn


wandb: Agent Starting Run: ndwtrweu with config:
wandb: 	batch_before_backwards: 20
wandb: 	batch_size: 500
wandb: 	dropout: 0.3
wandb: 	epochs: 50
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.00021092187045869445
wandb: 	len_data: 40228
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 20, 'batch_size': 500, 'dropout': 0.3, 'epochs': 50, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.00021092187045869445, 'len_data': 40228, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
50
{'batch_before_backwards': 20, 'batch_size': 500, 'dropout': 0.3, 'epochs': 50, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.00021092187045869445, 'len_data': 40228, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}


100%|██████████| 40228/40228 [00:13<00:00, 2923.44it/s]


filled =272093
empty  =49731
0.8454714378045143null_dog=0
SmoothL1Loss() Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.00021092187045869445
    maximize: False
    weight_decay: 0
)
smalll_lin_GRUNet(
  (batchnorm): BatchNorm1d(510, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=510, out_features=510, bias=True)
  (gru1): GRUCell(510, 64)
  (gru2): GRUCell(510, 64)
  (gru3): GRUCell(510, 64)
  (gru4): GRUCell(510, 64)
  (gru5): GRUCell(510, 64)
  (gru6): GRUCell(510, 64)
  (gru7): GRUCell(510, 64)
  (gru8): GRUCell(510, 64)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=512, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): Identity()
)


100%|██████████| 64/64 [00:32<00:00,  1.97it/s]it]
